In [57]:
#Import necessary modules
import impact as                             impt
import impact.plotting as                    implot
from impact.parsers import Parser as parser
import numpy as np
#SETTINGS FOR IMPACT. Change settings to alter calculations as required
impt.settings.perform_curve_fit = False
impt.settings.fit_type = 'gompertz'
impt.settings.outlier_cleaning_flag = True
impt.settings.max_fraction_replicates_to_remove = 0.3
impt.settings.std_deviation_cutoff=0.1
impt.settings.verbose = False
impt.settings.live_calculations = False
impt.settings.use_filtered_data = True
impt.settings.death_phase_hyperparameter = 1

import sys
if 'ipykernel' in sys.modules:
    from plotly.offline import init_notebook_mode
    from plotly.offline import iplot as plot
    from IPython.display import HTML

    # This is required for correct mathjax (latex) and documentation rendering
    HTML(
        """
        <script>
            var waitForPlotly = setInterval( function() {
                if( typeof(window.Plotly) !== "undefined" ){
                    MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });
                    MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);
                    clearInterval(waitForPlotly);
                }}, 250 );
        </script>
        """
    )
    init_notebook_mode(connected=True)
import plotly as py
from plotly import tools
import plotly.graph_objs as go
import colorlover as cl
import math
import plotly.io as pio
pio.templates.default = "plotly_white"
import plotly.colors
from plotly.subplots import make_subplots


colors = {'blues':['rgb(164,199,228)','rgb(119,171,214)','rgb(73,143,201)','rgb(49,113,165)','rgb(36,82,120)','rgb(23,52,76)'],
          'greens':['rgb(172,220,173)','rgb(130,203,133)','rgb(89,185,92)','rgb(63,152,66)','rgb(46,110,48)','rgb(28,67,29)'],
         'reds':['rgb(230,174,176)','rgb(218,133,137)','rgb(205,92,98)','rgb(187,57,64)','rgb(146,45,50)','rgb(104,32,35)'],
          #['rgb'+str(color) for color in plotly.colors.n_colors((254,224,210), (165,15,21), 6)],
         'pinks':['rgb(215,181,210)','rgb(195,144,188)','rgb(175,107,165)','rgb(147,79,137)','rgb(110,59,102)','rgb(74,40,69)'],
         'browns':['rgb(216,197,184)','rgb(196,168,148)','rgb(176,139,112)','rgb(150,111,83)','rgb(115,85,63)','rgb(80, 59, 44)'],
         'oranges': ['rgb(255,206,146)','rgb(255,189,109)','rgb(255,173,73)','rgb(255,157,36)','rgb(255, 140, 0)','rgb(219,120,0)'],
          'cyans': ['rgb(177,222,219)','rgb(139,205,201)','rgb(100,189,183)','rgb(70,163,157)','rgb(53,124,120)','rgb(37,87,84)'],
          'yellows': ['rgb(244,239,186)','rgb(239,231,151)','rgb(233,224,116)','rgb(228,216,82)','rgb(223,208,47)','rgb(200,185,31)',],
         'blacks':['rgb(182,182,182)','rgb(146,146,146)','rgb(109,109,109)','rgb(73,73,73)','rgb(36,36,36)','rgb(0,0,0)']}

colors_bg =  {'blues':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['blues']],
              'greens':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['greens']],
              'reds':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['reds']],
              'pinks':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['pinks']],
              'browns':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['browns']],
              'oranges':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['oranges']],
              'cyans': ['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['cyans']],
              'yellows': ['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['yellows']],
              'blacks':['rgba'+color.rstrip(')').lstrip('rgb')+',0.25)' for color in colors['blacks']]}
import pandas as pd
import pickle
with open('C:/Users/User/OneDrive - University of Toronto/Research/PhD_UofT/Data/Toggle_Switch_Project/Experiments/Screening/Autofluorescence_Models/gfp_models.pickle', 'rb') as handle:
    gfp_autofl_models = pickle.load(handle)

In [2]:
def bar_plot(replicate_trials=None,colors=None,plot_start=0, plot_end=-1, analytes_of_interest=['OD700','GFP','mCherry'], plot_width=900):
    max_norm=False
    plot_start = np.argmax(replicate_trials[0].t>plot_start)

    fig = make_subplots(rows=1, cols=2,
                    horizontal_spacing=0.125, subplot_titles=['Steady State Analyte Values', 'Max. Cell Specific Productivity'],
                    specs=[[{'secondary_y':True}, {'secondary_y':True}]])
    
    x = analytes_of_interest
    for index, replicate in enumerate(replicate_trials):
        y_ss = []
        y_ss_err = []
        dydt_max=[]
        dydt_max_err=[]
        for analyte in x:
            if analyte=='OD700':
                avg_analyte = replicate.avg.analyte_dict[analyte].data_vector
                std_analyte = replicate.std.analyte_dict[analyte].data_vector

                feature = 'specific_productivity'
                avg_rate = getattr(replicate.avg.analyte_dict[analyte], feature)
                std_rate = getattr(replicate.std.analyte_dict[analyte], feature)

            else:

                avg_analyte = replicate.avg.analyte_dict[analyte].od_normalized_data
                std_analyte = replicate.std.analyte_dict[analyte].od_normalized_data

    # Uncomment line under to get rate of change of cell specific flourescence.
    #                 rates = np.array([np.gradient(st.analyte_dict[analyte].od_normalized_data.data)/
    #                                  np.gradient(st.analyte_dict[analyte].time_vector)
    #                                  for st in replicate.single_trials 
    #                                  if str(st.trial_identifier.replicate_id) not in replicate.bad_replicates[analyte]+replicate.bad_replicates['OD700']])
    # This block gives cell specific rate of change of fluorescence i.e. 1/X*d[FL]/dt
#                 rates = np.array([np.gradient(st.analyte_dict[analyte].data_vector)/
#                                  np.gradient(st.analyte_dict[analyte].time_vector)/st.analyte_dict['OD700'].data_vector
#                                  for st in replicate.single_trials 
#                                  if str(st.trial_identifier.replicate_id) not in replicate.bad_replicates[analyte]+replicate.bad_replicates['OD700']])

#                 avg_rate = np.average(rates,axis=0)
#                 std_rate = np.std(rates, axis=0)
                avg_rate = getattr(replicate.avg.analyte_dict[analyte], 'specific_productivity')
                std_rate = getattr(replicate.std.analyte_dict[analyte], 'specific_productivity')

            y_ss.append(avg_analyte[-1])
            y_ss_err.append(std_analyte[-1])

                        
        fig.add_trace(go.Bar(x=[x[0]],
                             y=[y_ss[0]],
                             error_y=dict(type='data', array=[y_ss_err[0]], thickness=1.5, width=3, color='black'),
                             marker=dict(color=colors[index], line=dict(width=0)),
                             showlegend=True,# yaxis='y2',
                             legendgroup=str(replicate.trial_identifier.strain),
                             name=str(replicate.trial_identifier.strain)),
                     row=1, col=1, secondary_y=False)

        fig.add_trace(go.Bar(x=x[1:],
                             y=y_ss[1:],
                             error_y=dict(type='data', array=y_ss_err[1:], thickness=1.5, width=3, color='black'),
                             marker=dict(color=colors[index], line=dict(width=0)),
                             showlegend=False,# yaxis='y2',
                             legendgroup=str(replicate.trial_identifier.strain),
                             name=str(replicate.trial_identifier.strain)),
                     row=1, col=1, secondary_y=True)

  
        fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                          ticks='outside', ticklen=4, tickangle=0, nticks=8, title_font=dict(size=18, family='Myriad Pro', color='black'),
                          tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                          showgrid=False,zeroline=False)
        fig.update_annotations(font=dict(family='Myriad Pro', size=18, color='black'))
        width=plot_width
        fig.update_layout(height=410, width=width, legend_orientation='h', legend_y=-0.3, legend_x=0.15,
                          legend_font=dict(family='Myriad Pro', size=16, color='black'),bargap=0.4, bargroupgap=0.05)
        

        anchors = ['x1','x2']
        overlays = ['y1', 'y3']
        for ax_index, axis in enumerate(['yaxis2', 'yaxis4']):
            fig['layout'][axis] = dict(title='(a.u.)', title_standoff=0,
                                     title_font=dict(family='Myriad Pro', size=18, color='black'),
                                     showline=True, linewidth=1, linecolor='black', mirror=True,
                                     ticks='outside', ticklen=4, tickangle=0, overlaying=overlays[ax_index],
                                     tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                                     showgrid=True,zeroline=False,anchor=anchors[ax_index],side='right')

        for ax_index, axis in enumerate(['yaxis', 'yaxis3']):
            fig['layout'][axis] = dict(title='(a.u.)', title_standoff=0,
                                     title_font=dict(family='Myriad Pro', size=18, color='black'),
                                     showline=True, linewidth=1, linecolor='black', mirror=True,
                                     ticks='outside', ticklen=4, tickangle=0,
                                     tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                                     showgrid=True,zeroline=False,anchor=anchors[ax_index],side='left')

    return fig


def bar_plot_all(replicate_trials=None, analyte_of_interest='OD700',
                 plot_start=0, plot_end=-1, plot_width=900, norm=True, od_norm=True):
    title = ''
    if analyte_of_interest == 'OD700':
        title += analyte_of_interest+' '
        attr = 'data_vector'
    else:
        if od_norm:
            title += 'OD Normalized '+analyte_of_interest+' '
            attr = 'od_normalized_data'
        else:
            title += analyte_of_interest+' '
            attr='data_vector'
    
    max_ss = max([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
                 for replicate in replicate_trials])
    min_ss = min([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
                 for replicate in replicate_trials])

    if norm:
        title = title + 'scaled to the max value'

    #First separate by Degradation Tag - Observe effect of RBS
    trace_list = []
    for index,rbs_level in enumerate(['1','2','3','4','5']):
        reps=[]
        for rep in replicate_trials:
            if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
                ident = rep.trial_identifier.strain.name.replace('00','').split()
                if ident:
                    if ident[0][0] == rbs_level:
                        reps.append(rep)
        reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
        x = ['Degradation<br>Level 0', 'Degradation<br>Level 1', 'Degradation<br>Level 2', 'Degradation<br>Level 3']
        
        if norm:
            base = 0#min_ss
            scale = max_ss# - min_ss
        else:
            base=0
            scale=1

        y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
             for rep in reps]
        y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
             for rep in reps]
        
        if rbs_level=='1':
            showlegend=True
        else:
            showlegened=False
        if reps:
            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=colors['blues'][index], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(rbs_level),
                                 name='RBS Level: '+str(rbs_level)))

    strains = ['0000', 'Lac', 'Tet']
    temp_colors=[colors['blacks'][4], colors['oranges'][4], colors['pinks'][4]]
    for i,strain in enumerate(strains):
        x = ['Reference Strains']
        replicate = [rep for rep in replicate_trials if rep.trial_identifier.strain.name==strain]
        if replicate:
            replicate = replicate[0]
            y = [((getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]) - base)/(scale)]
            y_errs = [((getattr(replicate.std.analyte_dict[analyte_of_interest], attr)[-1]))/(scale)]
            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=temp_colors[i], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(strain),
                                 name=str(strain)))    
    width=plot_width
    fig1=go.Figure(data=trace_list)
    fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                      ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                      tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                      showgrid=False,zeroline=False)
    fig1.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                       title_font=dict(family='Myriad Pro',size=16, color='black'),
                       legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                       legend_font=dict(family='Myriad Pro', size=16, color='black'))
    fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
    fig1['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                                 title_font=dict(family='Myriad Pro', size=18, color='black'),
                                 showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                                 ticks='outside', ticklen=4, tickangle=0, type='log',
                                 tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                                 showgrid=True,zeroline=False,side='left')
    
    trace_list = []
    for index,deg_level in enumerate(['0','1','2','3']):
        reps=[]
        for rep in replicate_trials:
            if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
                ident = rep.trial_identifier.strain.name.replace('00','').split()
                if ident:
                    if ident[0][1] == deg_level:
                        reps.append(rep)
        reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
        x = ['RBS<br>Level 1', 'RBS<br>Level 2', 'RBS<br>Level 3', 'RBS<br>Level 4', 'RBS<br>Level 5']

        y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
             for rep in reps]
        y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
             for rep in reps]
        
        
        if deg_level=='0':
            showlegend=True
        else:
            showlegened=False
        if reps:
            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=colors['reds'][1:][index], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(deg_level),
                                 name='Degradation Level: '+str(deg_level)))
        
    strains = ['0000', 'Lac', 'Tet']
    temp_colors=[colors['blacks'][4], colors['oranges'][4], colors['pinks'][4]]
    for i,strain in enumerate(strains):
        x = ['Reference Strains']
        replicate = [rep for rep in replicate_trials if rep.trial_identifier.strain.name==strain]
        if replicate:
            replicate=replicate[0]
            y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
                 for rep in reps]
            y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
                 for rep in reps]

            trace_list.append(go.Bar(x=x,
                                 y=y,
                                 error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                                 marker=dict(color=temp_colors[i], line=dict(width=0)),
                                 showlegend=showlegend,# yaxis='y2',
                                 legendgroup=str(strain),
                                 name=str(strain)))    
    width=plot_width
    fig2=go.Figure(data=trace_list)
    fig2.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                      ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                      tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                      showgrid=False,zeroline=False)
    fig2.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                        title_font=dict(family='Myriad Pro',size=18, color='black'),
                      legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                      legend_font=dict(family='Myriad Pro', size=16, color='black'))
    fig2['layout']['title']['font'] = dict(family='Myriad Pro',size=18, color='black')

    fig2['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                                 title_font=dict(family='Myriad Pro', size=18, color='black'),
                                 showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                                 ticks='outside', ticklen=4, tickangle=0, type='log',
                                 tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                                 showgrid=True,zeroline=False,side='left')
    
    
    
    
    

    return fig1, fig2

In [3]:
data_format='tecan_OD_GFP_mCherry'
plate_type = '96 Wells'
file_name = 'pTSFus00XX_SparkEndPoint_24012021_D1.xlsx'
OD_data = parser.parse_raw_data(data_format=data_format,file_name=file_name,plate_type=plate_type)
expt = OD_data
expt.calculate()

gfp_model = gfp_autofl_models['film_spark'][90]
for replicate in expt.replicate_trials:
    if replicate.trial_identifier.strain.name.lower() != 'blank':
        for single_trial in replicate.single_trials:
            single_trial.analyte_dict['GFP'].pd_series.update(single_trial.analyte_dict['GFP'].pd_series - 
                                                              np.array([gfp_model(od_value) for od_value
                                                                        in single_trial.analyte_dict['OD700'].pd_series.values]))
            
expt.calculate()


Importing data from pTSFus00XX_SparkEndPoint_24012021_D1.xlsx...0.0s
Extracting timepoint data...Extracted time point data in 0.2s
Parsing time point list...Parsed 288 time points in 0.1s
Parsing analyte list...Parsed 96 single trials in 1700.1ms
Parsing single trial list...Parsed 24 replicates in 0.1s
Analyzing data...

C:\Users\User\anaconda3\envs\impact\lib\site-packages\numpy\core\fromnumeric.py:3622: RuntimeWarning:

Degrees of freedom <= 0 for slice

C:\Users\User\anaconda3\envs\impact\lib\site-packages\numpy\core\_methods.py:195: RuntimeWarning:

invalid value encountered in true_divide

C:\Users\User\anaconda3\envs\impact\lib\site-packages\numpy\core\_methods.py:226: RuntimeWarning:

invalid value encountered in double_scalars



Ran analysis in 1.7s

Analyzing data...Ran analysis in 1.6s



In [4]:
strains = ['0010', '0011', '0012', '0013',
           '0020', '0021', '0022', '0023',
          '0030', '0031', '0032', '0033',
          '0040', '0041', '0042', '0043',
          '0050', '0051', '0052', '0053']#, '0000','Tet','Lac']
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))

fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='mCherry', plot_width=600)
      
plot(fig1)
plot(fig2)

#fig1.write_image("fig1.svg")
#fig2.write_image("fig2.svg")

fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='mCherry', plot_width=600, norm=False)

plot(fig1)
plot(fig2)

strains_2 = ['0000','Tet','Lac']
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains_2]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))

fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='mCherry', plot_width=300, norm=False)
                     #colors=[colors['reds'][3], colors['greens'][3], colors['blues'][3],
                                      # colors['brown'], colors['orange'], colors['cyan']])
plot(fig1)
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, 
                       analyte_of_interest='mCherry', plot_width=600, norm=False, od_norm=False)
                     #colors=[colors['reds'][3], colors['greens'][3], colors['blues'][3],
                                      # colors['brown'], colors['orange'], colors['cyan']])
plot(fig1)
plot(fig2)
strains_2 = ['0000','Tet','Lac']
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains_2]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))

fig1,fig2=bar_plot_all(rep_list, plot_end=-1, plot_start=1, analyte_of_interest='mCherry', plot_width=300, norm=False, od_norm=False)
                     #colors=[colors['reds'][3], colors['greens'][3], colors['blues'][3],
                                      # colors['brown'], colors['orange'], colors['cyan']])
plot(fig1)


In [67]:
strains = ['0010', '0011', '0012', '0013',
           '0020', '0021', '0022', '0023',
          '0030', '0031', '0032', '0033',
          '0040', '0041', '0042', '0043',
          '0050', '0051', '0052', '0053']
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
norm = True
analyte_of_interest = 'mCherry'
title = ''
title += 'OD Normalized '+analyte_of_interest+' '
attr = 'data_vector'


max_ss = max([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
             for replicate in rep_list])
min_ss = min([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
             for replicate in rep_list])

if norm:
    title = title + 'scaled to the max value'

#First separate by Degradation Tag - Observe effect of RBS
trace_list = []
for index,rbs_level in enumerate(['1','2','3','4','5']):
    rbs_codes = {'1':'A', '2':'B', '3':'C', '4':'D', '5':'E'}
    reps=[]
    for rep in rep_list:
        if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
            ident = rep.trial_identifier.strain.name.replace('00','').split()
            if ident:
                if ident[0][0] == rbs_level:
                    reps.append(rep)
    reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
    x = ['No Tag', 'DAS Tag', 'DAS+2 Tag', 'LAA Tag']

    if norm:
        base = 0#min_ss
        scale = max_ss# - min_ss
    else:
        base=0
        scale=1

    y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
         for rep in reps]
    y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
         for rep in reps]

    if rbs_level=='1':
        showlegend=True
    else:
        showlegened=False
    if reps:
        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=py.colors.n_colors(py.colors.diverging.RdBu[-4], py.colors.diverging.RdBu[-2], 5, colortype='rgb')[index], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(rbs_level),
                             name='RBS "'+str(rbs_codes[rbs_level])+'"'))

width=700
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=450, width=625, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=18, color='black'),
                   legend_x=0.02, bargap=0.3, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=18, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=18, color='black')
fig1['layout']['yaxis'] = dict(title='Scaled Fluorescence (a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1.25, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig1)
trace_list = []
for index,deg_level in enumerate(['0','1','2','3']):
    deg_codes = {'0': 'No Tag', '1':'DAS Tag', '2':'DAS+2 Tag', '3':'LAA Tag'}
    reps=[]
    for rep in rep_list:
        if rep.trial_identifier.strain.name.lower() not in ['lac', 'tet', '0000']:
            ident = rep.trial_identifier.strain.name.replace('00','').split()
            if ident:
                if ident[0][1] == deg_level:
                    reps.append(rep)
    reps = sorted(reps, key=lambda rep: str(rep.trial_identifier.strain))
    x = ['RBS "A"', 'RBS "B"', 'RBS "C"', 'RBS "D"', 'RBS "E"']

    y = [(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1] - base)/(scale)
         for rep in reps]
    y_errs = [(getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1])/(scale)
         for rep in reps]


    if deg_level=='0':
        showlegend=True
    else:
        showlegened=False
    if reps:
        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=py.colors.n_colors(colors['oranges'][1], colors['oranges'][-1], 4, colortype='rgb')[::-1][index], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(deg_level),
                             name=deg_codes[deg_level]))

fig2=go.Figure(data=trace_list)
fig2.update_xaxes(showline=True, linewidth=1.25, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=18, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig2.update_layout(height=400, width=625, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=18, color='black'),
                   legend_x=0.02, bargap=0.3, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=18, color='black'))
fig2['layout']['title']['font'] = dict(family='Myriad Pro',size=18, color='black')
fig2['layout']['yaxis'] = dict(title='Scaled Fluorescence (a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1.25, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=18, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig2)

# fig1.write_image("fig00XX_RBS.svg", format='svg')
# fig2.write_image("fig00XX_deg.svg", format='svg')



In [68]:
strains = ['10', '11', '12', '13',
           '20', '21', '22', '23',
           '30', '31', '32', '33',
           '40', '41', '42', '43',
           '50', '51', '52', '53']
rep_list = [replicate for replicate in expt.replicate_trials if
            replicate.trial_identifier.strain.name[2:] in strains]
rep_list = sorted(rep_list, key=lambda rep: str(rep.trial_identifier.strain))
analyte_of_interest = 'mCherry'
attr = 'data_vector'
rbs_mean = {}
rbs_std = {}
deg_mean = {}
deg_std = {}
#First separate by Degradation Tag - Observe effect of RBS
trace_list = []
for index,rbs_level in enumerate(['1','2','3','4','5']):
    temp_reps = [rep for rep in rep_list if rep.trial_identifier.strain.name[2]==rbs_level]
    temp_reps = sorted(temp_reps, key=lambda rep:str(rep.trial_identifier.strain.name))
    max_ss = max([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
                 for replicate in temp_reps])    
    deg_mean = {**deg_mean, **{rep.trial_identifier.strain.name[2:]:
                               1/(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1]/max_ss)
                               for rep in temp_reps}}
    deg_std = {**deg_std, **{rep.trial_identifier.strain.name[2:]:
                               (getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1]/max_ss)/(getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1]/max_ss)**2
                               for rep in temp_reps}}
for index,deg_level in enumerate(['0','1','2','3']):
    temp_reps = [rep for rep in rep_list if rep.trial_identifier.strain.name[3]==deg_level]
    temp_reps = sorted(temp_reps, key=lambda rep:str(rep.trial_identifier.strain.name))
    min_ss = min([getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1]
                 for replicate in temp_reps])
    
    
    rbs_mean = {**rbs_mean, **{rep.trial_identifier.strain.name[2:]:
                               (getattr(rep.avg.analyte_dict[analyte_of_interest], attr)[-1]/min_ss)
                               for rep in temp_reps}}
    rbs_std = {**rbs_std, **{rep.trial_identifier.strain.name[2:]:
                               (getattr(rep.std.analyte_dict[analyte_of_interest], attr)[-1]/min_ss)
                               for rep in temp_reps}}
strains = ['10', '11', '12', '13',
           '20', '21', '22', '23',
           '30', '31', '32', '33',
           '40', '41', '42', '43',
           '50', '51', '52', '53']
rbs_mean_list = [rbs_mean[strain] for strain in strains]
rbs_std_list = [rbs_std[strain] for strain in strains]
deg_mean_list = [deg_mean[strain] for strain in strains]
deg_std_list = [deg_std[strain] for strain in strains]

mche_df = pd.DataFrame(data={'rbs_mean': rbs_mean_list,
                             'rbs_std': rbs_std_list,
                             'deg_mean': deg_mean_list,
                             'deg_std': deg_std_list,
                             'tet_code':strains})
mche_df = mche_df.set_index('tet_code')
display(mche_df)

,rbs_mean,rbs_std,deg_mean,deg_std
tet_code,,,,
10,1.000000,0.179167,1.000000,0.179167
11,1.203670,0.125560,6.791159,0.708413
12,3.666667,0.588442,15.576923,2.499848
13,1.000000,1.384437,278.437500,385.479264
20,1.433446,0.086374,1.000000,0.060256
21,1.000000,0.156082,11.717431,1.828880
22,3.743590,0.549141,21.869863,3.208055
23,3.250000,1.493039,122.807692,56.417454
30,1.095623,0.101453,1.000000,0.092599


In [70]:
mche_df.to_excel('C:\\Users\\User\\OneDrive - University of Toronto\\Research\\PhD_UofT\\Data\\Toggle_Switch_Project\\Experiments\\Screening\\pTSFusXXXX\\Speed\\mche_df.xlsx')

In [138]:
import colorlover as cl
from IPython.display import HTML

display(HTML(cl.to_html(py.colors.n_colors(py.colors.diverging.RdBu[3], py.colors.diverging.RdBu[1], 4, colortype='rgb'))))
display(HTML(cl.to_html(py.colors.n_colors(py.colors.diverging.RdBu[-4], py.colors.diverging.RdBu[-2], 5, colortype='rgb'))))
display(HTML(cl.to_html(py.colors.sequential.Greens[3:-2])))
display(HTML(cl.to_html(py.colors.sequential.Purp[2:-1])))
display(HTML(cl.to_html(colors['oranges'][0:1]+colors['oranges'][2:3]+colors['oranges'][4:])))
display(HTML(cl.to_html(colors['pinks'])))
display(HTML(cl.to_html(colors['blues'])))



In [119]:
color_scales[:,4]

IndexError: index 4 is out of bounds for axis 1 with size 4

In [114]:
color_scales

array([['rgb(209, 175, 232)', 'rgb(185, 152, 221)', 'rgb(159, 130, 206)',
        'rgb(130, 109, 186)'],
       ['rgb(255,206,146)', 'rgb(255,173,73)', 'rgb(255, 140, 0)',
        'rgb(219,120,0)'],
       ['rgb(161,217,155)', 'rgb(116,196,118)', 'rgb(65,171,93)',
        'rgb(35,139,69)'],
       ['rgb(244.0, 165.0, 130.0)', 'rgb(222.0, 118.0, 101.0)',
        'rgb(200.0, 71.0, 72.0)', 'rgb(178.0, 24.0, 43.0)'],
       ['rgb(146.0, 197.0, 222.0)',
        'rgb(108.33333333333334, 165.33333333333334, 205.33333333333334)',
        'rgb(70.66666666666667, 133.66666666666666, 188.66666666666666)',
        'rgb(33.0, 102.0, 172.0)']], dtype='<U63')

In [181]:
analyte_of_interest = 'mCherry'
strains = ['0000']
trace_list = []
temp_colors=[colors['blacks'][4]]
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
for i,strain in enumerate(strains):
    x = ['Reference Strains']
    replicate = [rep for rep in rep_list if rep.trial_identifier.strain.name==strain]
    if replicate:
        replicate = replicate[0]
        y = [(getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1])]
        y_errs = [np.sqrt(getattr(replicate.std.analyte_dict[analyte_of_interest], attr)[-1]**2)]


        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=temp_colors[i], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(strain),
                             name=str(strain)))    
width=300
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=16, color='black'),
                   legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=16, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
fig1['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig1)

In [186]:
analyte_of_interest = 'GFP'
strains = ['0000']
trace_list = []
temp_colors=[colors['blacks'][4]]
rep_list = [replicate for replicate in expt.replicate_trials if
            str(replicate.trial_identifier.strain.name) in strains]
for i,strain in enumerate(strains):
    x = ['Reference Strains']
    replicate = [rep for rep in rep_list if rep.trial_identifier.strain.name==strain]
    if replicate:
        replicate = replicate[0]
        y = [(getattr(replicate.avg.analyte_dict[analyte_of_interest], attr)[-1])]
        y_errs = [np.sqrt(getattr(replicate.std.analyte_dict[analyte_of_interest], attr)[-1]**2)]


        trace_list.append(go.Bar(x=x,
                             y=y,
                             error_y=dict(type='data', array=y_errs, thickness=1.5, width=3, color='black'),
                             marker=dict(color=temp_colors[i], line=dict(width=0)),
                             showlegend=showlegend,# yaxis='y2',
                             legendgroup=str(strain),
                             name=str(strain)))    
width=300
fig1=go.Figure(data=trace_list)
fig1.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                  ticks='outside', ticklen=4, tickangle=0, tickson='boundaries', title_font=dict(size=18, family='Myriad Pro', color='black'),
                  tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                  showgrid=False,zeroline=False)
fig1.update_layout(height=450, width=width, legend_orientation='h', legend_y=-0.2, 
                   title_font=dict(family='Myriad Pro',size=16, color='black'),
                   legend_x=0.02, bargap=0.2, bargroupgap=0.1, title=title,
                   legend_font=dict(family='Myriad Pro', size=16, color='black'))
fig1['layout']['title']['font'] = dict(family='Myriad Pro',size=16, color='black')
fig1['layout']['yaxis'] = dict(title='(a.u.)', title_standoff=0,
                             title_font=dict(family='Myriad Pro', size=18, color='black'),
                             showline=True, linewidth=1, linecolor='black', mirror=True, nticks=9,
                             ticks='outside', ticklen=4, tickangle=0, type='log',
                             tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 
                             showgrid=True,zeroline=False,side='left')
plot(fig1)